In [1]:
import pandas as pd 
import numpy as np

In [2]:
data = pd.read_pickle("crime_dataset.pkl")
data.head()

,date_reported,date_of_occurrence,time_of_occurrence,city,crime_code,crime_description,victim_age,victim_gender,weapon_used,crime_domain,police_deployed,case_closed,date_case_closed
report_number,,,,,,,,,,,,,
1,2020-01-02 00:00:00,2020-01-01 00:00:00,2020-01-01 01:11:00,Ahmedabad,576,IDENTITY THEFT,16,M,Blunt Object,Violent Crime,13,No,2050-01-01 00:00:00
2,2020-01-01 19:00:00,2020-01-01 01:00:00,2020-01-01 06:26:00,Chennai,128,HOMICIDE,37,M,Poison,Other Crime,9,No,2050-01-01 00:00:00
3,2020-01-02 05:00:00,2020-01-01 02:00:00,2020-01-01 14:30:00,Ludhiana,271,KIDNAPPING,48,F,Blunt Object,Other Crime,15,No,2050-01-01 00:00:00
4,2020-01-01 05:00:00,2020-01-01 03:00:00,2020-01-01 14:46:00,Pune,170,BURGLARY,49,F,Firearm,Other Crime,1,Yes,2020-04-29 05:00:00
5,2020-01-01 21:00:00,2020-01-01 04:00:00,2020-01-01 16:51:00,Pune,421,VANDALISM,30,F,Other,Other Crime,18,Yes,2020-01-08 21:00:00


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40160 entries, 1 to 40160
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date_reported       40160 non-null  datetime64[ns]
 1   date_of_occurrence  40160 non-null  datetime64[ns]
 2   time_of_occurrence  40160 non-null  datetime64[ns]
 3   city                40160 non-null  object        
 4   crime_code          40160 non-null  int64         
 5   crime_description   40160 non-null  object        
 6   victim_age          40160 non-null  int64         
 7   victim_gender       40160 non-null  object        
 8   weapon_used         40160 non-null  object        
 9   crime_domain        40160 non-null  object        
 10  police_deployed     40160 non-null  int64         
 11  case_closed         40160 non-null  object        
 12  date_case_closed    40160 non-null  datetime64[ns]
dtypes: datetime64[ns](4), int64(3), object(6)
memory us

In [4]:
df = data.copy()
df.sample(3)

,date_reported,date_of_occurrence,time_of_occurrence,city,crime_code,crime_description,victim_age,victim_gender,weapon_used,crime_domain,police_deployed,case_closed,date_case_closed
report_number,,,,,,,,,,,,,
28772,2023-04-16 01:00:00,2023-04-13 19:00:00,2023-04-14 02:28:00,Mumbai,499,CYBERCRIME,24,X,Firearm,Other Crime,17,No,2050-01-01 00:00:00
25211,2022-11-17 16:00:00,2022-11-16 10:00:00,2022-11-17 07:57:00,Kanpur,274,ARSON,43,M,Other,Fire Accident,9,No,2050-01-01 00:00:00
2024,2020-03-27 06:00:00,2020-03-25 07:00:00,2020-03-26 05:06:00,Visakhapatnam,213,SHOPLIFTING,26,F,Weapon,Other Crime,12,Yes,2020-05-29 06:00:00


In [5]:
df['hour_of_occurrence'] = df['time_of_occurrence'].dt.hour

In [6]:
df['day_of_week'] = df['date_of_occurrence'].dt.dayofweek

In [7]:
df['month_of_occurrence'] = df['date_of_occurrence'].dt.month

In [8]:
df['year_of_occurrence'] = df['date_of_occurrence'].dt.year

In [9]:
df['time_to_report'] = (df['date_reported'] - df['date_of_occurrence']).dt.days

In [10]:
df['time_to_close_case'] = (df['date_case_closed'] - df['date_of_occurrence']).dt.days

In [11]:
city_crime_rate = df['city'].value_counts(normalize=True)
df['city_crime_rate'] = df['city'].map(city_crime_rate)

In [12]:
df['city'] = df['city'].astype('category').cat.codes

In [13]:
df['crime_description_length'] = df['crime_description'].apply(len)

In [16]:
# Define a function to group crime domains into broader categories
def broad_grouping_function(crime_domain):
    if crime_domain in ['Violent Crime']:
        return 'Severe Crime'
    elif crime_domain in ['Fire Accident', 'Traffic Fatality']:
        return 'Accidental'
    else:
        return 'Other'

# Apply the function to create a new column
df['broad_crime_domain'] = df['crime_domain'].map(broad_grouping_function)

In [18]:
bins = [0, 12, 18, 60, 100]
labels = ['child', 'teen', 'adult', 'senior']
df['victim_age_group'] = pd.cut(df['victim_age'], bins=bins, labels=labels)

In [19]:
df['victim_gender'] = df['victim_gender'].map({'Male': 0, 'Female': 1})

In [20]:
avg_police_deployed = df.groupby('crime_code')['police_deployed'].transform('mean')
df['police_deployment_density'] = df['police_deployed'] / avg_police_deployed

In [21]:
df['case_closed'] = df['case_closed'].map({'Yes': 1, 'No': 0})

In [22]:
df['quick_case_closure'] = df['time_to_close_case'].apply(lambda x: 1 if x <= 30 else 0)

In [23]:
df['crime_time_interaction'] = df['crime_code'].astype(str) + '_' + df['hour_of_occurrence'].astype(str)

In [24]:
df['age_gender_interaction'] = df['victim_age_group'].astype(str) + '_' + df['victim_gender'].astype(str)

In [25]:
df['city_crime_count'] = df.groupby('city')['city'].transform('count')

In [26]:
df['avg_police_deployed_domain'] = df.groupby('crime_domain')['police_deployed'].transform('mean')

In [27]:
df.head()

,date_reported,date_of_occurrence,time_of_occurrence,city,crime_code,crime_description,victim_age,victim_gender,weapon_used,crime_domain,...,city_crime_rate,crime_description_length,broad_crime_domain,victim_age_group,police_deployment_density,quick_case_closure,crime_time_interaction,age_gender_interaction,city_crime_count,avg_police_deployed_domain
report_number,,,,,,,,,,,,,,,,,,,,,
1,2020-01-02 00:00:00,2020-01-01 00:00:00,2020-01-01 01:11:00,1,576,IDENTITY THEFT,16,NaN,Blunt Object,Violent Crime,...,0.045244,14,Severe Crime,teen,1.254507,0,576_1,teen_nan,1817,9.994073
2,2020-01-01 19:00:00,2020-01-01 01:00:00,2020-01-01 06:26:00,4,128,HOMICIDE,37,NaN,Poison,Other Crime,...,0.062077,8,Other,adult,0.969415,0,128_6,adult_nan,2493,10.015906
3,2020-01-02 05:00:00,2020-01-01 02:00:00,2020-01-01 14:30:00,15,271,KIDNAPPING,48,NaN,Blunt Object,Other Crime,...,0.018949,10,Other,adult,1.428571,0,271_14,adult_nan,761,10.015906
4,2020-01-01 05:00:00,2020-01-01 03:00:00,2020-01-01 14:46:00,21,170,BURGLARY,49,NaN,Firearm,Other Crime,...,0.055080,8,Other,adult,0.101124,0,170_14,adult_nan,2212,10.015906
5,2020-01-01 21:00:00,2020-01-01 04:00:00,2020-01-01 16:51:00,21,421,VANDALISM,30,NaN,Other,Other Crime,...,0.055080,9,Other,adult,1.844802,1,421_16,adult_nan,2212,10.015906


In [29]:
df.columns

Index(['date_reported', 'date_of_occurrence', 'time_of_occurrence', 'city',
       'crime_code', 'crime_description', 'victim_age', 'victim_gender',
       'weapon_used', 'crime_domain', 'police_deployed', 'case_closed',
       'date_case_closed', 'hour_of_occurrence', 'day_of_week',
       'month_of_occurrence', 'year_of_occurrence', 'time_to_report',
       'time_to_close_case', 'city_crime_rate', 'crime_description_length',
       'broad_crime_domain', 'victim_age_group', 'police_deployment_density',
       'quick_case_closure', 'crime_time_interaction',
       'age_gender_interaction', 'city_crime_count',
       'avg_police_deployed_domain'],
      dtype='object')

In [31]:
df.to_pickle("./FeatureCrimeData.pkl")